In [1]:
print("Let's Go")

Let's Go


### Imports

In [45]:
import pandas as pd
import numpy as np
import pyarrow as pa
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col, PandasUDFType, lit
from sklearn.ensemble import RandomForestClassifier
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType
from sklearn.metrics import accuracy_score
import joblib

In [46]:
spark = SparkSession.builder.appName("ML model").getOrCreate()

### Подготовка данных

In [47]:
data = pd.DataFrame({
    'feature1': np.random.randn(1000),
    'feature2': np.random.randn(1000),
    'feature3': np.random.randn(1000),
    'noise': np.random.choice([np.nan, 0, 1], 1000, p=[0.2, 0.4, 0.4])  # Добавляем NaN и шум
})

### Предобработка данных
Обработка пропущенных значений, масштабирование и уменьшение шума.


In [48]:
# Функция предобработки данных
def preprocess_data(df):
    # Заполнение пропущенных значений
    df['noise'] = df['noise'].fillna(df['noise'].median())
    # Нормализация признаков (пример)
    for col in ['feature1', 'feature2', 'feature3']:
        df[col] = (df[col] - df[col].mean()) / df[col].std()
    return df

# Применяем предобработку
data = preprocess_data(data)

In [49]:
data.head()

,feature1,feature2,feature3,noise
0,-1.313683,0.116888,-0.396188,1.0
1,-0.601685,-1.049042,-1.774992,1.0
2,-1.029277,-0.097142,-2.075136,0.0
3,-1.219959,-0.833503,-0.907069,1.0
4,0.882943,0.801273,1.514790,0.0


### Обучение модели (опционально)
Этап обучения может быть опущен, или вы можете использовать предобученную модель. Пример обучения модели включен для наглядности.

In [50]:
# Обучение простой модели RandomForest
X = data[['feature1', 'feature2']]
y = (data['feature3'] > 0).astype(int)  # Временная цель
model = RandomForestClassifier()
model.fit(X, y)

# Сохранение модели
joblib.dump(model, 'model.joblib')

['model.joblib']

### Загрузка модели

In [51]:
# Функция для загрузки модели
def load_model():
    return joblib.load('model.joblib')

### Определение Pandas UDF для предобработки и инференса модели
UDF будет включать как предобработку данных, так и инференс модели.

In [52]:
@pandas_udf('int')
def preprocess_and_predict_udf(feature1: pd.Series, feature2: pd.Series, noise: pd.Series) -> pd.Series:
    # Загрузка модели
    model = load_model()

    # Предобработка данных
    X = pd.DataFrame({'feature1': feature1, 'feature2': feature2, 'noise': noise})

    # Запуск инференса
    predictions = model.predict(X[['feature1', 'feature2']])
    return pd.Series(predictions)

### Создание Spark DataFrame
Преобразование pandas DataFrame в Spark DataFrame.

In [53]:
# Преобразование pandas DataFrame в Spark DataFrame
spark_df = spark.createDataFrame(data)

spark_df = spark_df.withColumn("ground_truth", (col("feature3") > 0).cast("int"))

In [54]:
spark_df.show()

+--------------------+--------------------+--------------------+-----+------------+
|            feature1|            feature2|            feature3|noise|ground_truth|
+--------------------+--------------------+--------------------+-----+------------+
| -1.3136834837162612| 0.11688845457066503| -0.3961883058960256|  1.0|           0|
| -0.6016848089643879|  -1.049042072000781| -1.7749923034772972|  1.0|           0|
| -1.0292768886001822|-0.09714204851222284| -2.0751355077379117|  0.0|           0|
|  -1.219959409873019| -0.8335027626304303| -0.9070690721924288|  1.0|           0|
|  0.8829430725419278|  0.8012726111587835|  1.5147900196280362|  0.0|           1|
| -2.0046987746532148|-0.01395306655154981|  1.1969216968494516|  1.0|           1|
|-0.39731578986254124|  0.8111434577801303|-0.01775789826526...|  0.0|           0|
|  1.0699556852111884| 0.11306710254079792|  0.4759820031444082|  1.0|           1|
| -0.7558235574095304|  0.6337715410445365| -2.5073597749519703|  0.0|      

### Применение Pandas UDF для предобработки и инференса

In [55]:
# Запуск предобработки и инференса с использованием pandas UDF
result_df = spark_df.withColumn("prediction", preprocess_and_predict_udf(col("feature1"), col("feature2"), col("noise")))
result_df.show()

+--------------------+--------------------+--------------------+-----+------------+----------+
|            feature1|            feature2|            feature3|noise|ground_truth|prediction|
+--------------------+--------------------+--------------------+-----+------------+----------+
| -1.3136834837162612| 0.11688845457066503| -0.3961883058960256|  1.0|           0|         0|
| -0.6016848089643879|  -1.049042072000781| -1.7749923034772972|  1.0|           0|         0|
| -1.0292768886001822|-0.09714204851222284| -2.0751355077379117|  0.0|           0|         0|
|  -1.219959409873019| -0.8335027626304303| -0.9070690721924288|  1.0|           0|         0|
|  0.8829430725419278|  0.8012726111587835|  1.5147900196280362|  0.0|           1|         1|
| -2.0046987746532148|-0.01395306655154981|  1.1969216968494516|  1.0|           1|         1|
|-0.39731578986254124|  0.8111434577801303|-0.01775789826526...|  0.0|           0|         0|
|  1.0699556852111884| 0.11306710254079792|  0.475

In [56]:
result_df.count()

1000

### Мониторинг качества

Вычисление точности модели или других показателей для мониторинга качества предсказаний.

In [57]:
# Собираем результаты для оценки
predictions = result_df.select("prediction").toPandas()
ground_truth = spark_df.select("ground_truth").toPandas()

# Вычисление точности
accuracy = accuracy_score(ground_truth, predictions)
print("Точность модели:", accuracy)

Точность модели: 1.0


### Еще один способ работы с pandas udf - GroupMap

Если задача требует передачи всей таблицы целиком для обработки одной моделью, то обычно лучше использовать Broadcast Variable для загрузки модели, что позволяет применить её к каждому фрагменту данных, не передавая DataFrame или модель в UDF.

Модель можно загрузить и сделать доступной для всех узлов Spark-кластера с помощью функции SparkContext.broadcast. Это создаст единственный экземпляр модели, который будет использоваться всеми узлами.

In [58]:
broadcast_model = spark.sparkContext.broadcast(model)

In [59]:
schema = StructType([
    StructField("prediction", IntegerType()),
    StructField("ground_truth", IntegerType()),
])

In [60]:
spark_df = spark_df.withColumn("group_column", lit(1))

In [61]:
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def group_predict_udf(pdf: pd.DataFrame) -> pd.DataFrame:
    # Доступ к переданной модели
    model = broadcast_model.value
    X = pdf.copy()
    X['prediction'] = model.predict(X[['feature1', 'feature2']])

    return X[['prediction', 'ground_truth']]

In [62]:
result_df = spark_df.groupBy("group_column").apply(group_predict_udf)
result_df.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/group_ops.py:104: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+----------+------------+
|prediction|ground_truth|
+----------+------------+
|         0|           0|
|         0|           0|
|         0|           0|
|         0|           0|
|         1|           1|
|         1|           1|
|         0|           0|
|         1|           1|
|         0|           0|
|         0|           0|
|         1|           1|
|         0|           0|
|         0|           0|
|         1|           1|
|         0|           0|
|         0|           0|
|         0|           0|
|         1|           1|
|         1|           1|
|         1|           1|
+----------+------------+
only showing top 20 rows



In [63]:
result_df.count()

1000

In [65]:
# Собираем результаты для оценки
predictions = result_df.select("prediction").toPandas()
ground_truth = result_df.select("ground_truth").toPandas()

# Вычисление точности
accuracy = accuracy_score(ground_truth, predictions)
print("Точность модели:", accuracy)

Точность модели: 1.0
